# 目的
bgeでのretrieveファイルの作成

- MAP@25: 
- recall: 

Please let me know if there are any mistakes.

# Setting

In [11]:
EXP_NAME = "e004-ret-bge-ft"
DATA_PATH = "../../data"
MODEL_NAME = "BAAI/bge-large-en-v1.5"

DATASET_NAME = EXP_NAME
MODEL_OUTPUT_PATH = f"../../eedi-mining-misconceptions-in-mathematics/trained_models/retriever/{EXP_NAME}"

RETRIEVE_NUM = 25  # TODO: 多くしてみる

# Import

In [12]:
import os

import polars as pl
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

In [45]:
NUM_PROC = 16

In [13]:
# import transformers
# import torch
# import sentence_transformers

# assert transformers.__version__ == "4.44.2"
# assert torch.__version__ == "2.3.1"
# assert sentence_transformers.__version__ == "3.1.0"

# Data Load

In [65]:
from datasets import load_dataset

train = (
    load_dataset(
        "csv",
        data_files="../../output/retriever/e003-ret-bge/e003-ret-bge-ret25-map0.1841-recall0.5506.csv",
        split="train",
    )
    .filter(
        lambda example: example["MisconceptionId"] is not None,
        num_proc=NUM_PROC,
    )
    .filter(  # anchor、positive、negativeの構成にするため、positiveとnegativeが一致している行を削除する
        lambda example: example["MisconceptionId"] != example["PredictMisconceptionId"],
        num_proc=NUM_PROC,
    )
)

Filter (num_proc=16):   0%|          | 0/109250 [00:00<?, ? examples/s]

Filter (num_proc=16):   0%|          | 0/109250 [00:00<?, ? examples/s]

In [61]:
print(train["AllText"][0])

Use the order of operations to carry out calculations involving powers BIDMAS \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ? Does not need brackets


In [62]:
print(train["MisconceptionName"][0])

Confuses the order of operations, believes addition comes before multiplication 


In [63]:
print(train["PredictMisconceptionName"][0])

Applies BIDMAS in strict order (does not realize addition and subtraction, and multiplication and division, are of equal priority)


# Fine-tuning BGE

In [36]:
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator

In [33]:
model = SentenceTransformer()  # CV: 0.1841439184198388

In [34]:
loss = MultipleNegativesRankingLoss(model)

In [35]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=MODEL_OUTPUT_PATH,
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name=EXP_NAME,  # Will be used in W&B if `wandb` is installed
)

In [78]:
train["PredictMisconceptionName"][:5]

['Applies BIDMAS in strict order (does not realize addition and subtraction, and multiplication and division, are of equal priority)',
 'Misunderstands order of operations in algebraic expressions',
 'Answers order of operations questions with brackets as if the brackets are not there',
 'May have made a calculation error using the order of operations',
 'Does not understand that the numerator and denominator of fractions represent groupings and have the same order of priority as brackets']

In [86]:
# 6. (Optional) Create an evaluator & evaluate the base model
dev_evaluator = TripletEvaluator(
    anchors=train["AllText"][:2],
    positives=train["MisconceptionName"][:2],
    negatives=train["PredictMisconceptionName"][:2],
    name=f"{MODEL_NAME}-dev",
)
dev_evaluator(model)

AttributeError: 'NoneType' object has no attribute 'tokenize'

In [7]:
from datasets import load_dataset

dataset = load_dataset("sentence-transformers/all-nli", "triplet")

README.md:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/38.4M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/782k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/810k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/557850 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/6584 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6609 [00:00<?, ? examples/s]

In [9]:
dataset["train"][0]

{'anchor': 'A person on a horse jumps over a broken down airplane.',
 'positive': 'A person is outdoors, on a horse.',
 'negative': 'A person is at a diner, ordering an omelette.'}

In [11]:
# model = SentenceTransformer("BAAI/bge-large-zh-v1.5") # CV: 0.06ぐらい
model = SentenceTransformer("BAAI/bge-large-en-v1.5")  # CV: 0.1841439184198388
# model = SentenceTransformer("all-MiniLM-L6-v2") # CV: 0.17806659878200218
# model = SentenceTransformer("nvidia/NV-Embed-v2", trust_remote_code=True)  # CV: OOMのため、試せていない
# model.max_seq_length = 32768
# model.tokenizer.padding_side = "right"
# model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)

train_long_vec = model.encode(
    train_long["AllText"].to_list(), normalize_embeddings=True
)
misconception_mapping_vec = model.encode(
    misconception_mapping["MisconceptionName"].to_list(), normalize_embeddings=True
)
print(train_long_vec.shape)
print(misconception_mapping_vec.shape)

(7476, 1024)
(2587, 1024)


In [12]:
# misconception_mapping_vecを保存する
os.makedirs(MODEL_OUTPUT_PATH, exist_ok=True)
np.save(f"{MODEL_OUTPUT_PATH}/misconception_mapping_vec.npy", misconception_mapping_vec)

In [13]:
train_cos_sim_arr = cosine_similarity(train_long_vec, misconception_mapping_vec)
train_sorted_indices = np.argsort(-train_cos_sim_arr, axis=1)

In [14]:
# example
def print_example(df: pl.DataFrame, sorted_indices: np.ndarray, idx: int) -> None:
    print(f"Query idx{idx}")
    print(df["AllText"][idx])
    print("\nCos Sim No.1")
    print(misconception_mapping["MisconceptionName"][int(sorted_indices[idx, 0])])
    print("\nCos Sim No.2")
    print(misconception_mapping["MisconceptionName"][int(sorted_indices[idx, 1])])

In [15]:
print_example(train_long, train_sorted_indices, 0)

Query idx0
Use the order of operations to carry out calculations involving powers BIDMAS \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ? \( 3 \times(2+4)-5 \)

Cos Sim No.1
Applies BIDMAS in strict order (does not realize addition and subtraction, and multiplication and division, are of equal priority)

Cos Sim No.2
Misunderstands order of operations in algebraic expressions


In [16]:
print_example(train_long, train_sorted_indices, 1)

Query idx1
Use the order of operations to carry out calculations involving powers BIDMAS \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ? \( 3 \times 2+(4-5) \)

Cos Sim No.1
Applies BIDMAS in strict order (does not realize addition and subtraction, and multiplication and division, are of equal priority)

Cos Sim No.2
Misunderstands order of operations in algebraic expressions


# Evaluate

In [17]:
train_long = train_long.with_columns(
    pl.Series(train_sorted_indices[:, :RETRIEVE_NUM].tolist()).alias(
        "PredictMisconceptionId"
    )
)

In [18]:
# https://www.kaggle.com/code/cdeotte/how-to-train-open-book-model-part-1#MAP@3-Metric
def map_at_25(predictions, labels):
    map_sum = 0
    for x, y in zip(predictions, labels):
        z = [1 / i if y == j else 0 for i, j in zip(range(1, 26), x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

In [19]:
map_at_25_score = map_at_25(
    train_long.filter(pl.col("MisconceptionId").is_not_null())[
        "PredictMisconceptionId"
    ],
    train_long.filter(pl.col("MisconceptionId").is_not_null())["MisconceptionId"],
)
map_at_25_score

0.1841439184198388

In [20]:
def recall(predictions, labels):
    acc_num = np.sum([1 for x, y in zip(predictions, labels) if y in x])
    return acc_num / len(predictions)


recall_score = recall(
    train_long.filter(pl.col("MisconceptionId").is_not_null())[
        "PredictMisconceptionId"
    ],
    train_long.filter(pl.col("MisconceptionId").is_not_null())["MisconceptionId"],
)
recall_score

0.5505720823798627

# Make Retrieved Train File

In [21]:
train_retrieved = (
    train_long.filter(
        pl.col(
            "MisconceptionId"
        ).is_not_null()  # TODO: Consider ways to utilize data where MisconceptionId is NaN.
    )
    .explode("PredictMisconceptionId")
    .with_columns(
        (pl.col("MisconceptionId") == pl.col("PredictMisconceptionId"))
        .cast(pl.Int64)
        .alias("target")
    )
    .join(
        misconception_mapping.with_columns(pl.all().name.prefix("Predict")),
        on="PredictMisconceptionId",
    )
)
train_retrieved.head()

QuestionId,ConstructName,SubjectName,QuestionText,CorrectAnswer,AnswerType,AnswerText,AllText,AnswerAlphabet,QuestionId_Answer,MisconceptionId,PredictMisconceptionId,target,MisconceptionId_right,MisconceptionName,PredictMisconceptionName
i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""Use the order of operations to…","""D""","""0_D""",1672,2306,0,2306,"""Applies BIDMAS in strict order…","""Applies BIDMAS in strict order…"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""Use the order of operations to…","""D""","""0_D""",1672,2586,0,2586,"""Misunderstands order of operat…","""Misunderstands order of operat…"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""Use the order of operations to…","""D""","""0_D""",1672,2488,0,2488,"""Answers order of operations qu…","""Answers order of operations qu…"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""Use the order of operations to…","""D""","""0_D""",1672,2518,0,2518,"""May have made a calculation er…","""May have made a calculation er…"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""Use the order of operations to…","""D""","""0_D""",1672,1316,0,1316,"""Does not understand that the n…","""Does not understand that the n…"


In [25]:
train_retrieved.write_csv(
    f"{MODEL_OUTPUT_PATH}/{EXP_NAME}-ret{RETRIEVE_NUM}-map{map_at_25_score:.4f}-recall{recall_score:.4f}.csv",
)

In [26]:
print(
    f"{MODEL_OUTPUT_PATH}/{EXP_NAME}-ret{RETRIEVE_NUM}-map{map_at_25_score:.4f}-recall{recall_score:.4f}.csv",
)

../../retrieved_data/e003-ret-bge-ret25_map0.1841_recall0.5506.csv


# Kaggle Upload

In [27]:
import os
import json

from kaggle.api.kaggle_api_extended import KaggleApi


def dataset_create_new(dataset_name: str, upload_dir: str):
    # if "_" in dataset_name:
    #     raise ValueError("datasetの名称に_の使用は禁止です")
    dataset_metadata = {}
    dataset_metadata["id"] = f"sinchir0/{dataset_name}"
    dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
    dataset_metadata["title"] = dataset_name
    with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")


print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

Create Dataset name:e003-ret-bge, output_dir:../../eedi-mining-misconceptions-in-mathematics/trained_models/retriever/e003-ret-bge
Starting upload for file misconception_mapping_vec.npy


100%|██████████| 10.1M/10.1M [00:04<00:00, 2.38MB/s]

Upload successful: misconception_mapping_vec.npy (10MB)
